In [3]:

import numpy as np
import matplotlib.pyplot as plt
import control as ctrl
import ipywidgets as widgets
from ipywidgets import interact, interactive, VBox, HBox, Button
from IPython.display import display, clear_output



roll_A = [[-17.5, -126],
          [1, 0]]
roll_B = [[-380],
          [0]]
roll_C = [[1, 0]]
roll_D = [[0]]



# pithc_A = [[-17.5, -126],
#           [1, 0]]

def define_state_space_system():
    """
    Define a second-order system:
    Example: ẍ + 3ẋ + 2x = u(t)
    Converted to state-space:
        A = [[0, 1], [-2, -3]]
        B = [[0], [1]]
        C = [[1, 0]]
        D = [[0]]
    """
    A = roll_A
    B = roll_B
    C = roll_C
    D = roll_D
    return ctrl.ss(A, B, C, D)



# PID controller in state-space form
def pid_controller_ss(Kp, Ki, Kd):
    # PID = Kd*s^2 + Kp*s + Ki / s
    num = [Kd, Kp, Ki]
    den = [1, 0]
    pid_tf = ctrl.TransferFunction(num, den)
    return ctrl.tf2ss(pid_tf)

# Connect both in state-space
def connect_controller_to_plant_ss(pid_ss, plant_ss):
    open_loop_ss = ctrl.series(pid_ss, plant_ss)
    closed_loop_ss = ctrl.feedback(open_loop_ss, 1)
    return closed_loop_ss

# Run simulation
def run_simulation(Kp, Ki, Kd, x0_val):
    plant_ss = define_state_space_system()
    pid_ss = pid_controller_ss(Kp, Ki, Kd)
    closed_loop_ss = connect_controller_to_plant_ss(pid_ss, plant_ss)

    # Simulate with initial condition
    n_states = closed_loop_ss.A.shape[0]
    x0 = np.zeros((n_states,))
    x0[0] = x0_val  # set first state

    T = np.linspace(0, 10, 1000)
    t, y = ctrl.step_response(closed_loop_ss, T=T, X0=x0)

    plt.figure(figsize=(8, 4))
    plt.plot(t, y, label=f'Initial state x0 = {x0_val}')
    plt.title("Step Response with PID (State-Space + Initial State)")
    plt.xlabel("Time [s]")
    plt.ylabel("Output")
    plt.grid(True)
    plt.legend()
    plt.show()


In [4]:


# Sliders
Kp_slider = widgets.FloatSlider(value=1.0, min=0.0, max=10.0, step=0.1, description='Kp')
Ki_slider = widgets.FloatSlider(value=1.0, min=0.0, max=10.0, step=0.1, description='Ki')
Kd_slider = widgets.FloatSlider(value=0.0, min=0.0, max=5.0, step=0.1, description='Kd')
x0_slider = widgets.FloatSlider(value=0.0, min=-5.0, max=5.0, step=0.1, description='Initial x₀')
restart_button = Button(description="Restart Simulation")
output = widgets.Output()

def on_restart_clicked(b):
    with output:
        clear_output(wait=True)
        run_simulation(Kp_slider.value, Ki_slider.value, Kd_slider.value, x0_slider.value)

restart_button.on_click(on_restart_clicked)

ui = VBox([
    HBox([Kp_slider, Ki_slider, Kd_slider]),
    HBox([x0_slider, restart_button]),
    output
])

display(ui)
on_restart_clicked(None)